In [51]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [52]:
# Setup splinter
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [53]:
#search page
# browser = Browser('firefox')
url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=pdr&search_distance=4#search=1~gallery~0~0'
header={}
browser.visit(url)
html = browser.html
website = soup(html, 'html.parser')

In [39]:
#collect all of the search results

results = website.find_all("div", class_="gallery-card")


In [40]:
results

[<div class="gallery-card"><div class="cl-gallery"><a class="main" href="https://sfbay.craigslist.org/sfc/off/d/san-francisco-for-lease-newly-renovated/7594322129.html"><div class="swipe"><div class="swipe-wrap"><div><span class="loading icom-"></span><img alt=""/></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div></div></div><div class="slider-back-arrow icom-"></div><div class="slider-forward-arrow icom-"></div></a><div class="dots"><span class="dot selected">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span><span class="dot">•</span></div></div><a class="titlestring" href="https://sfbay.craigslist.org/sfc/off/d/san-franci

In [41]:
results_info = []

for x in results:
    if x.find(class_='titlestring') is not None:
        title = x.find(class_='titlestring').text
        link = x.find(class_='titlestring')['href']
    else: title = None
    if x.find(class_='meta') is not None:
        info = x.find(class_='meta').text
    else: info = None
    if x.find(class_='priceinfo') is not None:
        price = x.find(class_='priceinfo').text
    else: price = None
    table = {
        "title":title,
        "link":link,
        "info":info,
        "price":price,
    }
    results_info.append(table)

In [42]:
df = pd.DataFrame(results_info)
df

,title,link,info,price
0,For Lease // Newly Renovated Legal Office/PDR ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,6h ago·SOMA / south beach,None
1,For Lease: Prime Retail / PDR Space With Flexi...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/27·12500ft2·mission district,None
2,For Lease // Newly Renovated Legal Office/PDR ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/6·SOMA / south beach,None
3,For Sale | Industrial Building w/ Secured Lot,https://sfbay.craigslist.org/sfc/off/d/san-fra...,6h ago·potrero hill,None
4,For Lease // Brand New Industrial Building w/ ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,7h ago·potrero hill,None
5,For Lease | Brick & Timber Warehouse - Cheap!!...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,7h ago·mission district,None
6,For Lease // Prime Industrial Warehouse w/ Ons...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/24·bayview,None
7,FOR LEASE: Ground Floor Office / Flex Space,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/10·1250ft2·mission district,"$4,000"
8,For Lease // Penthouse Office in Potrero Hill ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/6·potrero hill,None
9,For Sale | Industrial Complex w/ Secured Parking,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/30·potrero hill,None


In [47]:
df["info"] = df["info"].replace('[^a-zA-Z0-9/ -]+', " ", regex=True)

In [48]:
df.head()

,title,link,info,price
0,For Lease // Newly Renovated Legal Office/PDR ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,6h ago SOMA / south beach,None
1,For Lease: Prime Retail / PDR Space With Flexi...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/27 12500ft2 mission district,None
2,For Lease // Newly Renovated Legal Office/PDR ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/6 SOMA / south beach,None
3,For Sale | Industrial Building w/ Secured Lot,https://sfbay.craigslist.org/sfc/off/d/san-fra...,6h ago potrero hill,None
4,For Lease // Brand New Industrial Building w/ ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,7h ago potrero hill,None


In [5]:
meta = website.find("div", class_='meta')
meta
# meta = meta.find("span", class_="separator").remove()

In [8]:
#with many more results

url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=industrial&search_distance=4#search=1~gallery~0~0'
browser.visit(new_url)
html = browser.html
website = soup(html, 'html.parser')

results = website.find_all(class_="cl-search-result cl-search-view-mode-gallery")

results_info = []

for x in many_results:
    if x.find(class_='titlestring') is not None:
        title = x.find(class_='titlestring').text
        link = x.find(class_='titlestring')['href']
    else: new_title = None
    if x.find(class_='meta') is not None:
        info = x.find(class_='meta').text
    else: info = None
    if x.find(class_='priceinfo') is not None:
        price = x.find(class_='priceinfo').text
    else: price = None
    dictionary = {
        "title":title,
        "link":link,
        "info":info,
        "price":price,
    }
    results_info.append(dictionary)

In [10]:
big_df = pd.DataFrame(results_info)
# big_df["info"] = big_df["info"].replace("·","/")
# big_df[["date", "sqft","neighborhood"]] = big_df["info"].str.split("·",expand=True)
big_df.head()

""


In [55]:
#across multiple pages

url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=industrial&search_distance=4#search=1~gallery~0~0'
browser.visit(url)

for x in range(1,4):
    html = browser.html
    website = soup(html, 'html.parser')
    results = website.find_all(class_="cl-search-result cl-search-view-mode-gallery")

    all_results_info = []

    for x in results:
        if x.find(class_='titlestring') is not None:
            title = x.find(class_='titlestring').text
            link = x.find(class_='titlestring')['href']
        else: title = None
        if x.find(class_='meta') is not None:
            info = x.find(class_='meta').text
        else: info = None
        if x.find(class_='priceinfo') is not None:
            price = x.find(class_='priceinfo').text
        else: price = None
#         browser.find_by_css('button[class_="bd-button cl-next-page icon-only"]').click()
        browser.find_by_css(".cl-next-page").click()

        dictionary = {
            "title":title,
            "link":link,
            "info":info,
            "price":price,
        }
        results_info.append(dictionary)
        

In [50]:
results_info

[{'title': 'For Lease // Newly Renovated Legal Office/PDR Space',
  'link': 'https://sfbay.craigslist.org/sfc/off/d/san-francisco-for-lease-newly-renovated/7594322129.html',
  'info': '6h ago·SOMA / south beach',
  'price': None},
 {'title': 'For Lease: Prime Retail / PDR Space With Flexible Square Footage',
  'link': 'https://sfbay.craigslist.org/sfc/off/d/san-francisco-for-lease-prime-retail/7593943604.html',
  'info': '2/27·12500ft2·mission district',
  'price': None},
 {'title': 'For Lease // Newly Renovated Legal Office/PDR Space',
  'link': 'https://sfbay.craigslist.org/sfc/off/d/san-francisco-for-lease-newly-renovated/7586578133.html',
  'info': '2/6·SOMA / south beach',
  'price': None},
 {'title': 'For Sale | Industrial Building w/ Secured Lot',
  'link': 'https://sfbay.craigslist.org/sfc/off/d/san-francisco-for-sale-industrial/7594324556.html',
  'info': '6h ago·potrero hill',
  'price': None},
 {'title': 'For Lease // Brand New Industrial Building w/ Lab Potential',
  'link'

In [23]:
browser.quit()